In [1]:
import pandas as pd
from constants import HEADER_TEST_SAMPLE, DATA_DIR
test_df = pd.read_csv(DATA_DIR + 'Btest_sample_new.dat', sep='\t', header=None, names=HEADER_TEST_SAMPLE)
test_df.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


In [2]:
test_df[test_df.ad_id == 180]

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid


In [3]:
print(len(test_df.ad_id.drop_duplicates()))
print(len(test_df[[col for col in test_df.columns if col not in ('id', 'ad_id', 'bid')]].drop_duplicates()))

3750
3731


In [4]:
def get_most_likely_bid(bids):
    l = sorted(bids)
    return l[(len(l) -1) // 2]

In [5]:
min_test_sample = test_df[['ad_id', 'bid']].groupby('ad_id').agg(get_most_likely_bid)
min_test_sample['ad_id'] = min_test_sample.index
min_test_sample.head()

,bid,ad_id
ad_id,,
415,565,415
1030,225,1030
1090,27,1090
1383,44,1383
1496,38,1496


In [6]:
min_test_sample = min_test_sample.merge(test_df, on=['ad_id', 'bid'])
min_test_sample.head()

,bid,ad_id,id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow
0,565,415,23559,1547789504,1,117,1,-1,17240,"17592185782272,17592185782272,17592185782272,1...",all
1,225,1030,28420,1553148761,34,12,13,10763,1663,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33..."
2,27,1090,8483,1551955947,30,122,13,26955,1050,"69273464340480,69273464340480,69273464340480,6...",all
3,44,1383,24714,1545818583,30,122,13,181,21939,"281474976694272,281474976694272,28147497669427...",all
4,38,1496,28300,1546810488,40,224,13,8915,7512,"281474976694272,281474976694272,28147497669427...",area:11442


In [7]:
min_test_sample.describe()

,bid,ad_id,id,create_ts,sizes,industry_id,product_class,product_id,account_id
count,3750.000000,3750.000000,3750.000000,3.750000e+03,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000
mean,92.530133,549239.913067,19393.526133,1.546281e+09,32.599200,121.420533,7.466400,8896.075467,14771.312800
std,142.752633,241667.066698,11111.796202,1.411092e+07,17.696476,67.141375,6.062827,10700.041696,8603.159225
min,4.000000,415.000000,3.000000,1.428400e+09,1.000000,-1.000000,1.000000,-1.000000,80.000000
25%,30.000000,350181.000000,9923.500000,1.546933e+09,30.000000,63.000000,1.000000,-1.000000,6848.000000
50%,60.000000,726117.500000,19457.500000,1.551919e+09,30.000000,122.000000,12.000000,2555.000000,15054.500000
75%,109.750000,740968.750000,28835.500000,1.552896e+09,40.000000,172.000000,13.000000,18774.000000,22364.000000
max,2580.000000,746103.000000,38587.000000,1.553418e+09,66.000000,244.000000,18.000000,32864.000000,29742.000000


In [8]:
# min_test_sample.industry_id.value_counts()

In [9]:
# min_test_sample[HEADER_TEST_SAMPLE].to_csv(DATA_DIR + 'min_test_sample.dat', header=False, sep='\t', index=False)

In [10]:
test_pred = pd.read_csv(DATA_DIR + 'test_submission_origin.csv', sep=",", header=None,names=["id","y_pre"])

In [11]:
min_pred = test_pred.merge(min_test_sample[HEADER_TEST_SAMPLE], on=['id'])
min_pred.describe()

,id,y_pre,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,bid
count,3750.000000,3750.000000,3750.000000,3.750000e+03,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000
mean,19393.526133,117.475627,549239.913067,1.546281e+09,32.599200,121.420533,7.466400,8896.075467,14771.312800,92.530133
std,11111.796202,446.007004,241667.066698,1.411092e+07,17.696476,67.141375,6.062827,10700.041696,8603.159225,142.752633
min,3.000000,1.000000,415.000000,1.428400e+09,1.000000,-1.000000,1.000000,-1.000000,80.000000,4.000000
25%,9923.500000,3.300000,350181.000000,1.546933e+09,30.000000,63.000000,1.000000,-1.000000,6848.000000,30.000000
50%,19457.500000,10.400000,726117.500000,1.551919e+09,30.000000,122.000000,12.000000,2555.000000,15054.500000,60.000000
75%,28835.500000,36.200000,740968.750000,1.552896e+09,40.000000,172.000000,13.000000,18774.000000,22364.000000,109.750000
max,38587.000000,3917.800000,746103.000000,1.553418e+09,66.000000,244.000000,18.000000,32864.000000,29742.000000,2580.000000


In [12]:
min_pred[min_pred.y_pre <= 0]

,id,y_pre,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid


In [13]:
# 替换负值为测试集均值，TODO：使用训练集，区分不同id的覆盖范围
# 算法1. 79.8589分 base_rate: 0.39382972385368825
base_rate = min_pred[min_pred.y_pre >= 0]['y_pre'].sum() / min_pred[min_pred.y_pre >= 0]['bid'].sum()
base_rate

1.2695931847787243

In [14]:
import numpy as np
min_pred['rate'] = np.where(min_pred.y_pre > 0, min_pred.y_pre / min_pred.bid, base_rate)

In [15]:
submission = test_df.merge(min_pred[['ad_id', 'rate']])
submission['y_pre'] = (submission.bid * submission.rate).round()
submission[['id', 'y_pre']].to_csv(DATA_DIR + 'submission.csv', header=None, index=None)

In [16]:
submission.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid,rate,y_pre
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37,0.675862,25.0
1,1441,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",6,0.675862,4.0
2,9301,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",29,0.675862,20.0
3,11566,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",51,0.675862,34.0
4,16010,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",34,0.675862,23.0


In [17]:
# 算法2 79.8577分, base_rate: 0.39786288891783483，TODO：保留精确小数
# base_rate = min_pred[min_pred.y_pre > 0]['y_pre'].sum() / min_pred[min_pred.y_pre > 0]['bid'].sum()
# print(base_rate)
# min_pred['rate'] = np.where(min_pred.y_pre > 0, min_pred.y_pre / min_pred.bid, base_rate)
# submission = test_df.merge(min_pred[['ad_id', 'rate']])
# submission['y_pre'] = (submission.bid * submission.rate).round()
# submission[['id', 'y_pre']].to_csv(DATA_DIR + 'submission.csv', header=None, index=None)

In [19]:
# # 算法3 not fit for me?
# submission = test_df.merge(min_pred[['ad_id', 'y_pre']])
# submission['y_pre'] = (submission['y_pre'] + submission.bid / 10000).round(4)
# submission[['id', 'y_pre']].to_csv(DATA_DIR + 'submission.csv', header=None, index=None)
# submission.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid,y_pre
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37,19.6037
1,1441,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",6,19.6006
2,9301,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",29,19.6029
3,11566,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",51,19.6051
4,16010,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",34,19.6034
